In [7]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from sklearn.utils import class_weight, shuffle
from keras.losses import binary_crossentropy
from keras.applications.resnet50 import preprocess_input
import keras.backend as K
import tensorflow as tf
from sklearn.metrics import f1_score, fbeta_score
from keras.utils import Sequence
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from torchvision import datasets, models, transforms
import torch


WORKERS = 2
CHANNEL = 3

import warnings
warnings.filterwarnings("ignore")
IMG_SIZE = 512
NUM_CLASSES = 5
SEED = 77
TRAIN_NUM = 1000

In [18]:
def crop_image_from_gray(img,tol=7):
    print(img)
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        print(mask)
        print(mask.any(1))
        print(mask.any(0))
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

In [19]:
def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
#     plt.imshow(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
    plt.imshow(image)
    return image

In [ ]:
t_v=['train','val'];

for flag in t_v:
    for file_name in range(0,5):

        path= flag +"/"+str(file_name);
        path_all="images/"+ path;
        save_dir = "images/images_preprocess/"+path    
        names = os.listdir(path_all);
        
#         print(save_dir);

        for i in range(len(names)):
            path=f"{path_all}/{names[i]}";
            image = load_ben_color(path,sigmaX=10);
            cv2.imwrite(save_dir+'/'+names[i], image)
        print("done:",save_dir)
        

In [87]:
save_dir = "../input/aptos2019-blindness-detection/train_images_ben_preprocessing_sigmaX10"
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

for idx, row in tqdm_notebook(df_train.iterrows()):
    path=f"../input/aptos2019-blindness-detection/train_images/{row['id_code']}.png"
    image = load_ben_color(path,sigmaX=10)
    Image.fromarray(image).save(os.path.join(save_dir, "{}.png".format(row['id_code'])))

train
val
